<a href="https://colab.research.google.com/github/zhuzihan728/metal-binding-site-prediction/blob/main/metal_pos_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [227]:
pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [228]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [229]:
!tar -xvf /content/drive/MyDrive/uniprot_dataset.tar -C /content

ChEBI-IDs_for_metal_binding.tsv
POS_TRAIN_FULL.tsv
POS_TRAIN.tsv
NEG_TRAIN.fasta
POS_TRAIN.fasta
POS_TRAIN_FULL.fasta


In [230]:
from Bio import SeqIO
import pandas as pd
from sklearn.metrics import f1_score, recall_score, precision_score, precision_recall_fscore_support
import numpy as np

In [231]:
# for seq_record in SeqIO.parse("POS_TRAIN.fasta", "fasta"):
#   print(f"ID: {seq_record.id}")
#   print(f"SEQ: {repr(seq_record.seq)}")
#   print(f"Length: {len(seq_record)}")

#record_dict = SeqIO.to_dict(SeqIO.parse("test-dataset.fasta", "fasta"))
#record_dict

Generate fake predictions

In [232]:
# generates random predictions for protein sequences, format: (prot_accession, metal_id, pos)

def generate_one_predict(protein, metals):
  acc, seq = protein
  n = np.random.randint(10)
  pred = []

  metal = np.random.choice(metals, n, replace=False)
  rng = np.random.default_rng()
  pos = rng.choice(len(seq), n, replace=False)
  for x, y in zip(metal, pos):
    pred.append((acc, x, y))
  return pred



def generate_predictions(number): # number: number of prot_seqs to predict
  chebi_map = pd.read_csv('ChEBI-IDs_for_metal_binding.tsv', sep='\t')
  chebi_ids = chebi_map["ChEBI-ID"].unique()
  output = [] # list of dicts {"protein":, "chebi-id", "position", "score"}
  cnt = 0
  for seq_record in SeqIO.parse("POS_TRAIN.fasta", "fasta"):
    if cnt == number:
      break
    # print(f"ID: {seq_record.id}")
    # print(f"SEQ: {repr(seq_record.seq)}")
    # print(f"Length: {len(seq_record)}")
    protein = seq_record.id.split('|')[1], seq_record.seq
    output += generate_one_predict(protein, chebi_ids)
    cnt += 1
  return output
    



In [233]:
output = generate_predictions(10)
output

[('A0A023GPI8', 'CHEBI:24875', 55),
 ('A0A023GPI8', 'CHEBI:23378', 117),
 ('A0A023GPI8', 'CHEBI:49552', 122),
 ('A0A023GPI8', 'CHEBI:30408', 151),
 ('A0A023GPI8', 'CHEBI:29105', 174),
 ('A0A067XG71', 'CHEBI:24875', 24),
 ('A0A067XG71', 'CHEBI:60504', 70),
 ('A0A067XG71', 'CHEBI:21143', 31),
 ('A0A067XG71', 'CHEBI:190135', 4),
 ('A0A067XG71', 'CHEBI:25213', 144),
 ('A0A084JZF2', 'CHEBI:29035', 162),
 ('A0A084JZF2', 'CHEBI:60400', 0),
 ('A0A084JZF2', 'CHEBI:29103', 12),
 ('A0A084JZF2', 'CHEBI:16793', 239),
 ('A0A084JZF2', 'CHEBI:21137', 279),
 ('A0A096P8D3', 'CHEBI:18420', 104),
 ('A0A096P8D3', 'CHEBI:25213', 299),
 ('A0A096P8D3', 'CHEBI:49786', 45),
 ('A0A096P8D3', 'CHEBI:29105', 139),
 ('A0A096P8D3', 'CHEBI:29101', 358),
 ('A0A0D4WV12', 'CHEBI:29103', 3),
 ('A0A0D4WV12', 'CHEBI:29036', 217),
 ('A0A0D4WV12', 'CHEBI:29105', 221),
 ('A0A0D4WV12', 'CHEBI:49713', 67),
 ('A0A0D4WV12', 'CHEBI:60504', 30),
 ('A0A0D4WV12', 'CHEBI:29101', 122),
 ('A0A0F5HNH9', 'CHEBI:29103', 47),
 ('A0A0H2UMY0',

Encode metals to integers, 0 for non-binding amino acid

In [234]:
from sklearn import preprocessing
chebi_map = pd.read_csv('ChEBI-IDs_for_metal_binding.tsv', sep='\t')
labels = chebi_map["ChEBI-ID"].unique()
le = preprocessing.LabelEncoder()
encoded = le.fit_transform(labels) + 1

pd.DataFrame({'metal': labels, 'encoded': encoded})


,metal,encoded
0,CHEBI:48775,20
1,CHEBI:29108,17
2,CHEBI:48828,21
3,CHEBI:49415,22
4,CHEBI:23378,7
5,CHEBI:49552,23
6,CHEBI:29036,13
7,CHEBI:60240,27
8,CHEBI:190135,4
9,CHEBI:24875,8


In [235]:
# query to encoder the metal encodings

def encode_metal(metal, le):
  return le.transform([metal])[0] + 1

def encode_metal_list(metals, le):
  return le.transform(metals) + 1

print(encode_metal('CHEBI:177874', le))
print(encode_metal_list(['CHEBI:16793', 'CHEBI:177874'], le))


2
[1 2]


In [236]:
n = 10 # number of prot seqs

In [237]:
def retrive_one_label(acc, data, le):
  labels = data[data['Accession'] == acc]
  #print(labels)
  metals = encode_metal_list(list(labels['ChEBI-ID']), le)
  pos = list(labels['Position'])
  return metals, pos



gt_data = pd.read_csv('POS_TRAIN.tsv',sep='\t')
preds = pd.DataFrame(output,columns=['Accession', 'ChEBI-ID', 'Position'])

In [238]:
# import tracemalloc
# tracemalloc.start()

# snap1 = tracemalloc.take_snapshot()

def create_encoded_mask(gt_data, preds):

  y_true = np.array([], dtype = np.int8)
  y_pred = np.array([], dtype = np.int8)

  cnt = 0
  for seq_record in SeqIO.parse("POS_TRAIN.fasta", "fasta"):
    if cnt == n:
      break
    # print(f"ID: {seq_record.id}")
    # print(f"SEQ: {repr(seq_record.seq)}")
    # print(f"Length: {len(seq_record)}")
    acc = seq_record.id.split('|')[1]

    temp = np.zeros(len(seq_record), dtype = np.int8)
    metals, pos = retrive_one_label(acc, gt_data, le)
    temp[pos] = metals
    y_true = np.concatenate((y_true, temp),axis=None)
    
    temp = np.zeros(len(seq_record), dtype = np.int8)
    metals, pos = retrive_one_label(acc, preds, le)
    temp[pos] = metals
    y_pred = np.concatenate((y_pred, temp),axis=None)

    cnt += 1
  return y_true, y_pred

# snap2 = tracemalloc.take_snapshot()

# top_stats = snap2.compare_to(snap1, "lineno")

# for stat in top_stats:
#     print(stat)
y_true, y_pred = create_encoded_mask(gt_data, preds)
y_pred[8] = 12
print(y_true[:10])
print(y_pred[:10])

[ 0  0  0  0  0  0  0  0 12  0]
[ 0  0  0  0  0  0  0  0 12  0]


Calculate metrics

In [239]:

precision, recall, fbeta_score, support = precision_recall_fscore_support(y_true, y_pred, average=None, labels = [i for i in range(1, 30)])
print(f'precision: {precision}\n recall: {recall}\n fbeta_score: {fbeta_score}')


precision: [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.  ]
 recall: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.09090909
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
 fbeta_score: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.13333333
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
